Half-Adder Implementation

In [1]:
from qiskit import QuantumRegister, ClassicalRegister
from qiskit import Aer, execute, QuantumCircuit

# Building XOR function (say) xor(x,y,z) = (x, y, x+y+z (mod 2))
sub_circuit = QuantumCircuit(3, name='xor')
sub_circuit.cx(0,2)
sub_circuit.cx(1,2)
xor = sub_circuit.to_instruction()

# Building XOR_AND function (say) xor_and(x,y,z) = (x, y, z + x.y (mod 2))
# we are saying it AND by considering z = 0
oracl = QuantumCircuit(3, name='xor_and')
oracl.ccx(0,1,2)
xor_and = oracl.to_instruction()

q = QuantumRegister(4)
c = ClassicalRegister(4)
d = ClassicalRegister(4)
qc = QuantumCircuit(q,c)

qc.x(q[1]) # setting inputs 0 1 0 0

qc.append(xor,[q[0],q[1],q[2]])  # Sum +x +y
qc.append(xor_and,[0,1,3])       # Carry + x.y

for i in range(4) :
    qc.measure(q[i],c[3-i])   # We did this for reading classical bits from left to right

#Executing the circuit in the simulator
backend = Aer.get_backend('qasm_simulator')
qjob = execute(qc, backend, shots =1)
count = qjob.result().get_counts()
print(count)

qc.draw()

{'0110': 1}


┌──────┐┌──────────┐   ┌─┐      
q0_0: ─────┤0     ├┤0         ├───┤M├──────
      ┌───┐│      ││          │   └╥┘┌─┐   
q0_1: ┤ X ├┤1 xor ├┤1         ├────╫─┤M├───
      └───┘│      ││  xor_and │┌─┐ ║ └╥┘   
q0_2: ─────┤2     ├┤          ├┤M├─╫──╫────
           └──────┘│          │└╥┘ ║  ║ ┌─┐
q0_3: ─────────────┤2         ├─╫──╫──╫─┤M├
                   └──────────┘ ║  ║  ║ └╥┘
c0: 4/══════════════════════════╩══╩══╩══╩═
                                1  3  2  0